# Methanol-Synthese
Bedingungen: 

T=493;15K

p=50 bar

## Bilanzen
Stoffbilanzen

$\dot n_i = \dot n_{i, 0} - \sum_{j}{\nu_ij \xi_j}$ 

Energiebilanz

$0 = \dot Q + \sum\limits_{j}{\xi_j \Delta H_j}$

## Gleichgewichts-Konstanten

$exp \left(- \frac{\Delta G_i}{R T} \right) = K_p K_{\phi^{eq}} = K_x \prod_i \left( \frac{p}{p^0}\right)^{\nu_i}  K_{\phi^{eq}} =\prod_i (n_i)^{\nu_i}\left( \frac{p}{p^0}\right)^{\sum_i \nu_i}(n)^{-\sum_i \nu_i}   K_{\phi^{eq}}$

Idealer Gas, $K_{\phi^{eq}}=1$ 

Van't Hoff, $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2}$

~ Geringe Veränderung der Reaktionsenthalpie mit der Temperatur: $\frac{d ln K}{dT} = -\frac{\Delta H}{R T^2} \sim \Rightarrow ln \left(\frac{K}{K'} \right) = -\frac{\Delta H^0}{R}\left(\frac{1}{T} - \frac{1}{T'} \right)$

$K_{(493,15K)} = K_{(298,15K)} \times exp \left[-\frac{\Delta H^0}{R}\left(\frac{1}{493,15 K} - \frac{1}{298,15 K} \right)\right] = K_x \prod\limits_{i}{\left( \frac{p}{p^0}\right)^{\nu_i} }= K_x \left( \frac{p}{p^0}\right)^{\sum\limits_{i}{\nu_i}}$ 

In [43]:
from scipy import optimize
import numpy as np

p = 50. # bar
temp = 273.15 + 220. # K

namen = ['CO', 'H2', 'CO2', 'H2O', 'CH3OH']

n0co = 50.
n0h2 = 100.
n0co2 = 10.
n0h2o = 0.
n0ch3oh = 0.

ne = np.array([n0co, n0h2, n0co2, n0h2o, n0ch3oh])

nuij = np.array([[-1, -2, 0, 0, +1] ,
                 [0, -3, -1, +1, +1], 
                 [-1, +1, +1, -1, 0]]).T

delta_h_1_298 = (-(-110.53) -2*0 + -200.94)*1000
delta_h_2_298 = (-(-393.50) -3*0 + -200.94 + -241.83)*1000
delta_h_3_298 = (-(-110.53) -(-241.83) + -393.50 + 0)*1000

delta_g_1_298 = (-(-137.20) -2*0 + -162.32)*1000
delta_g_2_298 = (-(-394.36) -3*0 + -162.32 + -228.62)*1000
delta_g_3_298 = (-(-137.20) -(-228.62) + -394.36 + 0)*1000

k1_298 = np.exp(-delta_g_1_298/(8.314*298.15))
k2_298 = np.exp(-delta_g_2_298/(8.314*298.15))
k3_298 = np.exp(-delta_g_3_298/(8.314*298.15))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_493 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/493.15 - 1/298.15 ))
k2_493 = k2_298 * np.exp(-delta_h_2_298/8.314*( 1/493.15 - 1/298.15 ))
k3_493 = k3_298 * np.exp(-delta_h_3_298/8.314*( 1/493.15 - 1/298.15 ))

for i, f in enumerate([delta_h_1_298, delta_h_2_298, delta_h_3_298]):
    print('Delta H_' + str(i+1) + '(298.15K)=' + str(f/1000.) + 'kJ/mol')

print('\n')
for i, f in enumerate([k1_493, k2_493, k3_493]):
    print('K' + str(i+1) + '(493K)=' + str(f))
print('\n')
    
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k1_493 * (nco * nh2**2) + \
            nch3oh * (p/1.)**-2 * (nco + nh2 + nch3oh)**-(-2)
    
    return [f1, f2, f3, f4]

n0 = np.array([n0co, n0h2, n0ch3oh])
x0 = np.append(n0, [0.])
#n0 = np.array([21.125, 42.25, 28.875])
#x0 = np.append(n0, [28.875])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + ne[[0,1,4]].reshape([3,1]) + nuij[:,0][[0,1,4]].reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für nur einzige Reaktion (ohne CO2):\n')
for i, f in enumerate(sol.x[:2]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('n_' + namen[-1] + '= ' + str(sol.x[2]) + ' mol')

# Lösung des einfacheren Falls in schwierigerem Fall einwenden.
def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nco2 = x_vec[2]
    nh2o = x_vec[3]
    nch3oh = x_vec[4]
    xi1 = x_vec[5]
    xi2 = x_vec[6]
    xi3 = x_vec[7]
    
    f1 = -nco + n0co - xi1 +0 -xi3
    f2 = -nh2  + n0h2 -2*xi1 -3*xi2 +xi3
    f3 = -nco2 + n0co2 +0 -xi2 +xi3
    f4 = -nh2o + n0h2o +0 +xi2 -xi3
    f5 = -nch3oh + n0ch3oh +xi1 +xi2 -0
    f6 = -k1_493 * (nco * nh2**2) + \
        nch3oh * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f7 = -k2_493 * (nco2 * nh2**3) + \
        nch3oh * nh2o * (p/1.)**-2 * (nco + nh2 + nco2 + nh2o + nch3oh)**-(-2)
    f8 = -k3_493 * (nco * nh2o) + \
        nco2 * nh2 * (p/1.)**0 * (nco + nh2 + nco2 + nh2o + nch3oh)**-0
    
    return [f1, f2, f3, f4, f5, f6, f7, f8]

n0 = np.array([sol.x[0], sol.x[1], n0co2, 0, sol.x[2]])
x0 = np.append(n0, [sol.x[3], 0, 0])
#n0 = np.array([1.79111141, 3.58222282, 10., 0, 48.20888859])
#x0 = np.append(n0, [48.20888859, 0, 0])
sol = optimize.root(fun, x0)

f_final = - sol.x[:5].reshape([5,1]) + ne.reshape([5,1]) + nuij.dot(sol.x[5:].reshape([3,1]))

print('\n\n')
print('T=493.15K, p=50 bar')
print('Lösung für alle drei Reaktionen, mit CO2:\n')
for i, f in enumerate(sol.x[:5]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
    
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

Delta H_1(298.15K)=-90.41kJ/mol
Delta H_2(298.15K)=-49.27kJ/mol
Delta H_3(298.15K)=-41.14kJ/mol


K1(493K)=0.0137296251237
K2(493K)=9.71591716437e-05
K3(493K)=141.310644085


    fjac: array([[ -1.75351909e-02,  -3.00618805e-10,  -8.45288537e-19,
         -9.99846247e-01],
       [ -3.77569225e-01,  -9.25957684e-01,  -1.07472894e-16,
          6.62176683e-03],
       [ -1.28777561e-01,   5.25265893e-02,  -9.90278848e-01,
          2.25848635e-03],
       [  9.16815324e-01,  -3.73956314e-01,  -1.39096382e-01,
         -1.60790038e-02]])
     fun: array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        -2.84217094e-14])
 message: 'The solution converged.'
    nfev: 17
     qtf: array([  4.96165502e-11,  -3.44442016e-13,  -1.12244387e-13,
         7.99109510e-13])
       r: array([  5.70281791e+01,  -2.32538544e+01,  -8.64947836e+00,
         4.30818643e-02,   1.07996300e+00,   5.72836364e-02,
         2.22931540e+00,   1.00981658e+00,  -9.66612171e-01,
        -3.07588250e-0

In [47]:
# Vorsicht! K-Wert im Aspen Plus
k1_493 = 0.00267615835

def fun(x_vec):    
    nco = x_vec[0]
    nh2 = x_vec[1]
    nch3oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nco + n0co - xi1
    f2 = -nh2  + n0h2 -2*xi1
    f3 = -nch3oh + n0ch3oh +xi1
    f4 = -k1_493 * (nco * nh2**2) + nch3oh * (p/1.)**-2 * (nco + nh2 + nch3oh)**-(-2)
    
    return [f1, f2, f3, f4]
print fun([21.125, 42.25, 28.875, 28.875 ])
print fun([11.4243863316, 22.8487726633, 38.5756136684, 38.5756136684])

[42.25/21.125, 22.8487726633/11.4243863316]

[0.0, 0.0, 0.0, -2.6252250719152244]
[0.0, -1.000017846308765e-10, 0.0, 65.9260634809681]


[2.0, 2.0000000000087534]

# SVN 13.6 (S. 485)
$C_2H_4(g) + H_2 O(g) \rightleftharpoons C_2H_5OH(g)$

In [25]:
p = 35. # bar
temp = 523.15 # K

namen = ['C2H4', 'H2O', 'C2H5OH']

n0c2h4 = 2.
n0h2o = 5.
n0c2h5oh = 0.

ne = np.array([n0c2h4, n0h2o, n0c2h5oh])

nuij = np.array([[-1, -1, +1]]).T

delta_h_1_298 = (-(+52.467) -(-241.826) + -234.906)*1000

delta_g_1_298 = (-(+68.420) -(-228.620) + -168.200)*1000

k1_298 = np.exp(-delta_g_1_298/(8.314*298.15))

# Angenommen, dass Delta_H fast im Interval zwischen 298.15K und 443.15K 
# unabhängig von der Temperatur sei...
k1_523 = k1_298 * np.exp(-delta_h_1_298/8.314*( 1/523.15 - 1/298.15 ))

print('\n')
for i, f in enumerate([k1_523]):
    print('K' + str(i+1) + '(523.15K)=' + str(f))
print('\n')


def fun(x_vec):    
    nc2h4 = x_vec[0]
    nh2o = x_vec[1]
    nc2h5oh = x_vec[2]
    xi1 = x_vec[3]
    
    f1 = -nc2h4 + n0c2h4 -xi1
    f2 = -nh2o  + n0h2o -xi1
    f3 = -nc2h5oh + n0c2h5oh +xi1
    f4 = -k1_523 * (nc2h4 * nh2o) + \
            nc2h5oh * (p/1.)**-1. * (nc2h4 + nh2o + nc2h5oh)**(-(-1))
    
    return [f1, f2, f3, f4]

n0 = np.array([(0.233), n0h2o, n0c2h5oh])
x0 = np.append(n0, [0.233])
sol = optimize.root(fun, x0)
f_final = - sol.x[:3].reshape([3,1]) + n0.reshape([3,1]) + nuij.reshape([3,1])*sol.x[-1]

print(sol)
print('\n\n')
print('Zustand der Optimisierungs-Funktionen\n')
print(f_final)

print('\n\n')
print('T=' + str(temp) +'K, p=' + str(p) +'bar')
for i, f in enumerate(sol.x[:-1]):
    print('n_' + namen[i] + '= ' + str(f) + ' mol')
print('x1= ' + str(sol.x[-1]))



K1(523.15K)=0.00932292186355


    fjac: array([[ -9.99223948e-01,   2.10438579e-10,  -1.04114753e-10,
         -3.93891191e-02],
       [  5.01145052e-04,  -9.99919060e-01,  -8.25469011e-10,
         -1.27130629e-02],
       [ -6.22922226e-03,  -2.01224058e-03,  -9.87413727e-01,
          1.58023035e-01],
       [  3.88902089e-02,   1.25628023e-02,  -1.58158566e-01,
         -9.86567583e-01]])
     fun: array([  5.55111512e-17,  -1.66533454e-16,   0.00000000e+00,
         0.00000000e+00])
 message: 'The solution converged.'
    nfev: 11
     qtf: array([  2.24060002e-13,   7.19101664e-14,  -8.98671873e-13,
         5.61057783e-12])
       r: array([  1.00077665e+00,   5.01569423e-04,  -6.31454054e-03,
         9.98252337e-01,   1.00008095e+00,  -2.03805403e-03,
         9.99104324e-01,   1.01274671e+00,  -9.75274364e-01,
        -2.33946887e-01])
  status: 1
 success: True
       x: array([ 1.62885912,  4.62885912,  0.37114088,  0.37114088])



Zustand der Optimisierungs-Funktionen
